<div style="display: flex; justify-content: space-between; align-items: center; padding: 8px 16px; background: #F8F9FA; border-bottom: 2px solid #E0E0E0; margin: 0; line-height: 1;">
    <div style="font-size: 14px; color: #666;">
        <span style="font-weight: bold; color: #333;">{SOURCE_PLATFORM} → Databricks Migration</span>
        <span style="margin-left: 8px; color: #999;">|</span>
        <span style="margin-left: 8px;">06 - Closeout</span>
    </div>
    <div style="display: flex; align-items: center; gap: 8px;">
        <img src="https://cdn.simpleicons.org/snowflake/29B5E8" width="24" height="24"/>
        <span style="color: #999; font-size: 16px;">→</span>
        <img src="https://cdn.simpleicons.org/databricks/FF3621" width="24" height="24"/>
    </div>
</div>

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# Decommissioning and Retirement

## Overview

Decommissioning {SOURCE_PLATFORM} after migration is **project closure**, not ongoing operations. This lesson covers how to safely retire your source platform while preserving necessary data, shutting down integrations in the correct sequence, maintaining compliance requirements, verifying cost elimination, and formally closing the migration engagement.

A structured decommissioning process prevents unexpected costs, ensures audit compliance, and provides clear closure for stakeholders.

## Learning Objectives

By the end of this lesson, you will be able to:
- Create compliant backup and archival procedures for {SOURCE_PLATFORM} data
- Execute safe connector and integration shutdown sequences
- Preserve metadata for audit and compliance requirements
- Verify complete cost wind-down
- Conduct final engagement review and formal closure

## Decommissioning Timeline

Decommissioning should follow a phased approach with clear validation gates between stages. Rushing this process can result in data loss or compliance violations.

<br />
<div class="mermaid">
gantt
    title {SOURCE_PLATFORM} Decommissioning Timeline
    dateFormat  YYYY-MM-DD
    section Stabilization
    Monitor Databricks Production    :done, s1, 2024-01-01, 14d
    Verify No Rollback Needed         :done, s2, 2024-01-08, 7d
    section Backup
    Archive Historical Data           :active, b1, 2024-01-15, 7d
    Preserve Metadata & Logs          :active, b2, 2024-01-15, 7d
    section Shutdown
    Disable Write Connections         :s1, 2024-01-22, 3d
    Shutdown Read Connections         :s2, 2024-01-25, 3d
    Decommission Warehouses           :s3, 2024-01-28, 2d
    section Closure
    Verify Zero Usage & Costs         :c1, 2024-01-30, 5d
    Final Review & Sign-off           :milestone, c2, 2024-02-04, 0d
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### Decommissioning Phases

| Phase | Duration | Exit Criteria | Risk of Early Exit |
|-------|----------|---------------|--------------------|
| **Stabilization** | 2-4 weeks | No P1/P2 incidents, business sign-off | High - premature shutdown |
| **Backup & Archive** | 1 week | All data archived, compliance verified | Critical - data loss |
| **Connection Shutdown** | 1 week | Zero active connections, integration tests pass | Medium - unexpected dependencies |
| **Resource Decommission** | 3-5 days | All resources deleted, $0 daily spend | Low - minor cost leakage |
| **Formal Closure** | 1 week | Stakeholder sign-off, documentation complete | Low - administrative only |

## {SOURCE_PLATFORM} Backup and Archival

Before decommissioning, create compliant archives of data that must be retained for audit, compliance, or historical reference purposes.

<br />
<div class="mermaid">
flowchart TB
    subgraph IDENTIFY["<b>Identify Archive Scope</b>"]
        COMPLIANCE["Compliance<br/>Requirements"]
        HISTORICAL["Historical<br/>Reference Data"]
        AUDIT["Audit<br/>Trail Logs"]
    end
    subgraph EXTRACT["<b>Extract & Preserve</b>"]
        TABLES["Export Tables<br/>to Parquet/Delta"]
        METADATA["Export Metadata<br/>(DDL, configs)"]
        LOGS["Archive Query<br/>& Access Logs"]
    end
    subgraph STORE["<b>Archival Storage</b>"]
        S3["AWS S3<br/>Glacier Deep Archive"]
        AZURE["Azure Blob<br/>Archive Tier"]
        GCS["GCS Archive<br/>Storage Class"]
    end
    IDENTIFY --> EXTRACT --> STORE
    STORE --> VERIFY["Verify Archive<br/>Integrity & Accessibility"]
    VERIFY --> |"Success"| APPROVE["✓ Approved for<br/>Decommissioning"]
    VERIFY --> |"Failure"| EXTRACT
    style APPROVE fill:#e8f5e9,stroke:#4caf50
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### Archive Categories

| Category | What to Archive | Retention Period | Format |
|----------|----------------|------------------|--------|
| **Compliance Data** | Tables with PII, financial records | 7-10 years | Encrypted Parquet/Delta |
| **Historical Snapshots** | Point-in-time data for reference | 3-5 years | Parquet on archive storage |
| **Metadata** | DDL, schemas, object definitions | Permanent | JSON + SQL scripts |
| **Audit Logs** | Query history, access logs | 1-3 years | JSON/CSV |
| **Configuration** | Warehouse settings, role definitions | Permanent | JSON/YAML |

### Archive Export Script

<div class="code-block" data-language="python">
# Example: Archive {SOURCE_PLATFORM} data to cloud storage before decommissioning
from datetime import datetime
import hashlib

# Archive configuration
archive_config = {
    "archive_root": "s3://company-compliance-archive/{SOURCE_PLATFORM}/",
    "encryption": "AES-256",
    "storage_class": "GLACIER_DEEP_ARCHIVE",
    "retention_years": 7
}

# Export table to archive storage
def archive_table(schema: str, table: str, archive_path: str):
    """
    Export {SOURCE_PLATFORM} table to Parquet archive with verification.
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_location = f"{archive_path}/{schema}/{table}/{timestamp}/"
    
    # Read from {SOURCE_PLATFORM} via Databricks Partner Connect or JDBC
    df = (
        spark.read
        .format("{SOURCE_PLATFORM}")
        .option("dbtable", f"{schema}.{table}")
        .load()
    )
    
    # Calculate row count and checksum for verification
    row_count = df.count()
    
    # Write to archive with metadata
    (
        df.write
        .mode("overwrite")
        .option("compression", "snappy")
        .parquet(archive_location)
    )
    
    # Create manifest for verification
    manifest = {
        "source_schema": schema,
        "source_table": table,
        "archive_location": archive_location,
        "archive_timestamp": timestamp,
        "row_count": row_count,
        "retention_until": f"{datetime.now().year + archive_config['retention_years']}"
    }
    
    return manifest

# Archive all compliance-critical schemas
compliance_schemas = [
    "finance",
    "customer_data",
    "audit_logs"
]

for schema in compliance_schemas:
    tables = spark.sql(f"SHOW TABLES IN {schema}").collect()
    for table_row in tables:
        table = table_row.tableName
        manifest = archive_table(schema, table, archive_config["archive_root"])
        print(f"Archived {schema}.{table}: {manifest['row_count']} rows")
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-python.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'python';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

<div style="border-left: 4px solid #ff9800; background: #fff3e0; padding: 16px 20px; border-radius: 4px; margin: 16px 0;">
    <div style="display: flex; align-items: flex-start; gap: 12px;">
        <span style="font-size: 24px;">⚠️</span>
        <div>
            <strong style="color: #e65100; font-size: 1.1em;">Archive Verification is Critical</strong>
            <p style="margin: 8px 0 0 0; color: #333;">
                Always verify archives are readable and complete before decommissioning. Test restoration of at least 5% of archived tables. Document archive locations and access procedures for future compliance audits.
            </p>
        </div>
    </div>
</div>

## Metadata Preservation

Preserve {SOURCE_PLATFORM} metadata for audit trails, compliance investigations, and historical reference. This includes object definitions, access patterns, and query history.

### Metadata Export Checklist

| Metadata Type | Export Method | Storage Format | Use Case |
|---------------|---------------|----------------|----------|
| **DDL Definitions** | `SHOW CREATE TABLE` for all objects | SQL scripts | Schema reconstruction |
| **Object Lineage** | Dependency graphs and views | JSON/GraphML | Impact analysis |
| **Access Policies** | Role grants, row/column policies | JSON + SQL | Compliance audit |
| **Query History** | Last 90 days of queries | Parquet | Performance analysis |
| **Usage Statistics** | Warehouse usage, user activity | CSV/JSON | Cost justification |
| **Cost Reports** | Historical spend by workload | CSV | ROI documentation |

### Metadata Export Script

<div class="code-block" data-language="sql">
-- Export all table DDL from {SOURCE_PLATFORM}
-- Run this in {SOURCE_PLATFORM} and save output to archival storage

-- Get all table DDL
SELECT 
    table_catalog || '.' || table_schema || '.' || table_name AS full_table_name,
    'CREATE TABLE ' || table_catalog || '.' || table_schema || '.' || table_name AS ddl_statement,
    table_type,
    created AS table_created_at,
    last_altered AS table_last_modified
FROM information_schema.tables
WHERE table_schema NOT IN ('INFORMATION_SCHEMA', 'ACCOUNT_USAGE')
ORDER BY table_catalog, table_schema, table_name;

-- Export query history (last 90 days)
SELECT 
    query_id,
    query_text,
    database_name,
    schema_name,
    user_name,
    warehouse_name,
    execution_status,
    total_elapsed_time,
    rows_produced,
    bytes_scanned,
    start_time,
    end_time
FROM table(information_schema.query_history(
    dateadd('days', -90, current_timestamp()),
    current_timestamp()
))
ORDER BY start_time DESC;

-- Export access history for compliance
SELECT 
    user_name,
    direct_objects_accessed,
    base_objects_accessed,
    objects_modified,
    query_start_time
FROM table(information_schema.access_history(
    dateadd('days', -90, current_timestamp()),
    current_timestamp()
))
ORDER BY query_start_time DESC;
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Connector and Integration Shutdown Sequence

Shutdown connections in a safe sequence to prevent disruptions. Start with write connections, then read connections, and finally administrative access.

<br />
<div class="mermaid">
flowchart TB
    START["Begin Shutdown"] --> AUDIT["Audit All Connections"]
    AUDIT --> WRITE["<b>Phase 1: Write Connections</b><br/>Disable ETL jobs, data loaders"]
    WRITE --> VERIFY1{"Verify Zero<br/>Write Activity"}
    VERIFY1 --> |"Active writes detected"| INVESTIGATE1["Investigate &<br/>Migrate Remaining"]
    INVESTIGATE1 --> WRITE
    VERIFY1 --> |"No writes"| READ["<b>Phase 2: Read Connections</b><br/>Disable BI tools, API consumers"]
    READ --> VERIFY2{"Verify Zero<br/>Read Activity"}
    VERIFY2 --> |"Active reads detected"| INVESTIGATE2["Investigate &<br/>Update Connection Strings"]
    INVESTIGATE2 --> READ
    VERIFY2 --> |"No reads"| WAREHOUSE["<b>Phase 3: Warehouses</b><br/>Suspend/delete compute"]
    WAREHOUSE --> ADMIN["<b>Phase 4: Admin Access</b><br/>Revoke user credentials"]
    ADMIN --> FINAL["<b>Phase 5: Account Closure</b><br/>Contact vendor to close account"]
    style VERIFY1 fill:#fff3e0,stroke:#ff9800
    style VERIFY2 fill:#fff3e0,stroke:#ff9800
    style FINAL fill:#e8f5e9,stroke:#4caf50
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### Shutdown Sequence Checklist

| Phase | Actions | Validation | Rollback Time |
|-------|---------|------------|---------------|
| **1. Write Connections** | Disable ETL jobs, data loaders, streaming ingestion | Monitor for 48 hours | 1 hour |
| **2. Read Connections** | Update BI tools, downstream consumers, APIs | Monitor for 1 week | 4 hours |
| **3. Warehouses** | Suspend compute, delete warehouses | Verify $0 compute spend | 24 hours |
| **4. Storage** | Delete databases (after archive verified) | Verify $0 storage spend | N/A - irreversible |
| **5. Account** | Revoke all user access, close account | Confirm account closed | N/A - irreversible |

### Connection Audit Script

<div class="code-block" data-language="sql">
-- Audit all active connections to {SOURCE_PLATFORM}
-- Run this BEFORE starting shutdown sequence

-- Active sessions by user
SELECT 
    user_name,
    COUNT(*) AS active_sessions,
    MAX(created_on) AS last_connection,
    LISTAGG(DISTINCT client_application_id, ', ') AS applications
FROM table(information_schema.sessions(
    dateadd('days', -7, current_timestamp()),
    current_timestamp()
))
GROUP BY user_name
HAVING COUNT(*) > 0
ORDER BY active_sessions DESC;

-- Recent write operations
SELECT 
    user_name,
    database_name,
    schema_name,
    COUNT(*) AS write_operations,
    MAX(start_time) AS last_write
FROM table(information_schema.query_history(
    dateadd('hours', -24, current_timestamp()),
    current_timestamp()
))
WHERE query_type IN ('INSERT', 'UPDATE', 'DELETE', 'MERGE', 'COPY')
GROUP BY user_name, database_name, schema_name
ORDER BY last_write DESC;

-- Identify integration points
SELECT DISTINCT
    client_application_id,
    client_application_version,
    COUNT(DISTINCT user_name) AS unique_users,
    MAX(created_on) AS last_seen
FROM table(information_schema.sessions(
    dateadd('days', -30, current_timestamp()),
    current_timestamp()
))
GROUP BY client_application_id, client_application_version
ORDER BY last_seen DESC;
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

<div style="border-left: 4px solid #f44336; background: #ffebee; padding: 16px 20px; border-radius: 4px; margin: 16px 0;">
    <div style="display: flex; align-items: flex-start; gap: 12px;">
        <span style="font-size: 24px;">🚫</span>
        <div>
            <strong style="color: #c62828; font-size: 1.1em;">Never Skip Connection Audits</strong>
            <p style="margin: 8px 0 0 0; color: #333;">
                Unexpected connections can cause production incidents. Always audit for at least 7 days before each shutdown phase. Document all discovered connections and verify migration to Databricks.
            </p>
        </div>
    </div>
</div>

## Cost Wind-Down Verification

Track cost elimination to verify successful decommissioning and capture migration ROI. {SOURCE_PLATFORM} costs should trend to zero over the decommissioning period.

<br />
<div class="mermaid">
%%{init: {'theme':'base', 'themeVariables': { 'primaryColor':'#e3f2fd','primaryTextColor':'#000','primaryBorderColor':'#1976d2','lineColor':'#1976d2','secondaryColor':'#e8f5e9','tertiaryColor':'#fff3e0'}}}%%
xychart-beta
    title "{SOURCE_PLATFORM} Cost Wind-Down"
    x-axis [Week 1, Week 2, Week 3, Week 4, Week 5, Week 6]
    y-axis "Cost ($)" 0 --> 50000
    line [45000, 42000, 35000, 18000, 5000, 0]
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### Cost Verification Checklist

| Cost Component | Expected Trend | Verification Method | Target |
|----------------|----------------|---------------------|--------|
| **Compute (Warehouses)** | Immediate drop after suspension | Daily billing reports | $0 within 3 days |
| **Storage** | Gradual decrease as data deleted | Storage usage metrics | $0 within 30 days |
| **Networking** | Drop with reduced queries | Data transfer costs | $0 within 1 week |
| **Serverless/Auto-scaling** | Drop with reduced load | Serverless credit usage | $0 within 1 week |
| **Support Subscription** | Manual cancellation required | Contact vendor | Cancelled |
| **Marketplace Integrations** | Manual review required | Audit marketplace apps | All removed |

### Cost Monitoring Dashboard

<div class="code-block" data-language="sql">
-- Monitor {SOURCE_PLATFORM} cost trends during decommissioning
-- Run daily to track cost elimination

SELECT 
    DATE_TRUNC('day', usage_date) AS day,
    service_type,
    SUM(credits_used) AS total_credits,
    SUM(credits_used * credit_price) AS total_cost_usd
FROM table(information_schema.warehouse_metering_history(
    dateadd('days', -30, current_timestamp()),
    current_timestamp()
))
GROUP BY day, service_type
ORDER BY day DESC, total_cost_usd DESC;

-- Identify remaining cost sources
SELECT 
    warehouse_name,
    SUM(credits_used) AS credits_last_7_days,
    MAX(end_time) AS last_activity,
    CASE 
        WHEN MAX(end_time) < DATEADD('day', -7, CURRENT_TIMESTAMP()) THEN 'Safe to delete'
        WHEN MAX(end_time) < DATEADD('day', -2, CURRENT_TIMESTAMP()) THEN 'Review for deletion'
        ELSE 'Still active - investigate'
    END AS recommendation
FROM table(information_schema.warehouse_metering_history(
    dateadd('days', -7, current_timestamp()),
    current_timestamp()
))
GROUP BY warehouse_name
HAVING credits_last_7_days > 0
ORDER BY credits_last_7_days DESC;
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Final Engagement Review and Closure

Formal closure includes stakeholder review, ROI documentation, lessons learned, and transition to steady-state operations.

<br />
<div class="mermaid">
flowchart LR
    subgraph REVIEW["<b>Final Review</b>"]
        METRICS["Collect Migration<br/>Metrics & KPIs"]
        ROI["Calculate<br/>ROI & Savings"]
        LESSONS["Document<br/>Lessons Learned"]
    end
    subgraph SIGNOFF["<b>Stakeholder Sign-off</b>"]
        EXEC["Executive<br/>Sponsors"]
        TECH["Technical<br/>Leads"]
        BIZ["Business<br/>Owners"]
    end
    subgraph TRANSITION["<b>Transition</b>"]
        OPS["Handoff to<br/>Platform Ops Team"]
        CLOSE["Close Project<br/>Artifacts"]
        CELEBRATE["Celebrate<br/>Success! 🎉"]
    end
    REVIEW --> SIGNOFF --> TRANSITION
    style CELEBRATE fill:#e8f5e9,stroke:#4caf50
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### Final Review Checklist

| Category | Deliverables | Owner | Due |
|----------|-------------|-------|-----|
| **Migration Metrics** | Total workloads migrated, success rate, timeline | PMO | Week 1 |
| **ROI Analysis** | Cost savings, performance improvements, TCO | Finance + Engineering | Week 1 |
| **Technical Documentation** | Architecture diagrams, runbooks, SOPs | Engineering | Week 2 |
| **Lessons Learned** | What went well, challenges, recommendations | All teams | Week 2 |
| **Stakeholder Sign-off** | Executive sponsor approval, business validation | PMO | Week 3 |
| **Decommission Confirmation** | {SOURCE_PLATFORM} shutdown verified, $0 costs | Engineering | Week 3 |
| **Knowledge Transfer** | Training completion, documentation handoff | Enablement | Week 4 |
| **Project Closure** | Close Jira, archive repos, final retrospective | PMO | Week 4 |

### Migration ROI Template

<div class="code-block" data-language="markdown">
# Migration to Databricks: ROI Summary

## Executive Summary
Migration from {SOURCE_PLATFORM} to Databricks completed on [DATE]. This document summarizes the business value delivered.

## Migration Scope
- **Workloads Migrated:** [X] ETL pipelines, [Y] analytics workloads, [Z] ML models
- **Data Volume:** [X] TB migrated, [Y] tables
- **Timeline:** [Start Date] to [End Date] ([X] months)
- **Team Size:** [X] engineers, [Y] business stakeholders

## Cost Savings (Annual)
| Category | {SOURCE_PLATFORM} Cost | Databricks Cost | Savings |
|----------|----------------------|-----------------|--------|
| **Compute** | $XXX,XXX | $XXX,XXX | $XX,XXX (XX%) |
| **Storage** | $XX,XXX | $XX,XXX | $X,XXX (XX%) |
| **Support/Licenses** | $XX,XXX | $XX,XXX | $X,XXX (XX%) |
| **Engineering Overhead** | $XX,XXX | $XX,XXX | $X,XXX (XX%) |
| **TOTAL ANNUAL SAVINGS** | - | - | **$XXX,XXX (XX%)** |

## Performance Improvements
- **Query Performance:** XX% faster on average
- **ETL Runtime:** XX% reduction in pipeline duration
- **Time to Insight:** XX% faster analytics delivery
- **Developer Productivity:** XX% reduction in development time

## Business Value
- **New Capabilities Unlocked:** Real-time analytics, ML workflows, data lakehouse architecture
- **Compliance & Governance:** Unified access control, audit logging, data lineage
- **Scalability:** Elastic compute, support for 10x data growth
- **Innovation Velocity:** Faster experimentation, self-service analytics

## Lessons Learned
### What Went Well
- [Key success 1]
- [Key success 2]

### Challenges & Resolutions
- [Challenge 1]: [How we solved it]
- [Challenge 2]: [How we solved it]

### Recommendations for Future Migrations
- [Recommendation 1]
- [Recommendation 2]

## Next Steps
- {SOURCE_PLATFORM} account closed: [DATE]
- Transition to steady-state operations: [DATE]
- Continue platform optimization and enablement

## Sign-off
- **Executive Sponsor:** [Name] - [Date]
- **Technical Lead:** [Name] - [Date]
- **Business Owner:** [Name] - [Date]
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-markdown.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'markdown';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

<div style="border-left: 4px solid #4caf50; background: #e8f5e9; padding: 16px 20px; border-radius: 4px; margin: 16px 0;">
    <div style="display: flex; align-items: flex-start; gap: 12px;">
        <span style="font-size: 24px;">✓</span>
        <div>
            <strong style="color: #2e7d32; font-size: 1.1em;">Celebrate the Achievement</strong>
            <p style="margin: 8px 0 0 0; color: #333;">
                Cloud migrations are significant technical and organizational achievements. Take time to recognize team contributions, document successes, and share learnings across the organization.
            </p>
        </div>
    </div>
</div>

## Summary and Conclusion

Successful decommissioning requires systematic execution across five key areas:

### Decommissioning Checklist

- [ ] **Stabilization:** Monitor Databricks production for 2-4 weeks, confirm no rollback needed
- [ ] **Backup & Archive:** Export compliance data, metadata, and audit logs to archival storage
- [ ] **Archive Verification:** Test restore procedures, document archive locations
- [ ] **Connection Shutdown:** Disable write connections → read connections → warehouses → admin access
- [ ] **Connection Audits:** Monitor for 7+ days between shutdown phases
- [ ] **Cost Verification:** Confirm $0 compute, storage, and subscription costs
- [ ] **ROI Documentation:** Calculate savings, performance improvements, business value
- [ ] **Lessons Learned:** Document successes, challenges, and recommendations
- [ ] **Stakeholder Sign-off:** Obtain executive, technical, and business approval
- [ ] **Account Closure:** Contact {SOURCE_PLATFORM} to formally close account
- [ ] **Knowledge Transfer:** Hand off to platform operations team
- [ ] **Project Closure:** Archive project artifacts, celebrate team success

### Decommissioning Anti-Patterns to Avoid

| Anti-Pattern | Risk | Mitigation |
|--------------|------|------------|
| **Premature Shutdown** | Data loss, production incidents | Wait 2-4 weeks post-migration, monitor actively |
| **Skip Archive Verification** | Compliance violations | Test restore of 5%+ of archives |
| **No Connection Audits** | Unexpected downtime | Audit continuously for 7+ days |
| **Ignore Cost Monitoring** | Budget overruns | Track daily until $0 confirmed |
| **Skip Lessons Learned** | Repeat mistakes | Document within 2 weeks of closure |

### Transition to Operations

With decommissioning complete, the migration project ends and **platform operations** begin:

- **Platform Operations:** Ongoing monitoring, optimization, cost management ([5.1 - Platform Operations and Cost Management]($./../05 - Enable/5.1 - Platform Operations and Cost Management))
- **User Enablement:** Continuous training, adoption programs ([5.3 - Developer Enablement and Adoption]($./../05 - Enable/5.3 - Developer Enablement and Adoption))
- **Innovation:** Leverage Databricks capabilities for new use cases

### Final Thoughts

Decommissioning is not merely "turning off the old system"—it's the final validation that your migration delivered value. Proper decommissioning:

- **Confirms migration success** by eliminating legacy costs
- **Protects the business** through compliant archival and audit trails
- **Captures ROI** with documented savings and performance improvements
- **Enables future migrations** through lessons learned and best practices

Congratulations on completing your migration to Databricks!

---

### Return to Course Overview

Return to the [**Course README**]($./../README.md) for the complete migration curriculum.

<div style="color: #FF3621; font-weight: bold; font-size: 2em; margin-bottom: 12px;">COURSE DEVELOPER (remove before publishing)</div>

### Template Customization

**Placeholders to replace:**
- `{SOURCE_PLATFORM}` - Source platform name (e.g., Snowflake, Redshift, BigQuery)

**Platform-specific customization:**
- **Archive export scripts:** Adapt SQL queries for platform-specific information schema
- **Cost monitoring queries:** Replace with platform-specific cost/metering tables
- **Connection audit:** Update session monitoring queries for platform
- **Shutdown sequence:** Add platform-specific shutdown considerations (e.g., Snowflake resource monitors, Redshift cluster resize, BigQuery slot reservations)
- **Metadata preservation:** Include platform-specific objects (e.g., Snowflake tasks, Redshift WLM queues, BigQuery authorized views)

**Content expansion suggestions:**
- Add platform-specific compliance requirements (e.g., SOC2, HIPAA, GDPR)
- Include vendor-specific account closure procedures
- Add real-world decommissioning timelines from customer migrations
- Include platform-specific cost optimization during wind-down (e.g., reducing warehouse sizes before deletion)

**Code examples to add:**
- Platform-specific archive automation scripts
- Connection string update scripts for common BI tools
- Cost trend visualization queries
- Automated compliance report generation


<!-- PREVIOUS ONLY -->
<div style="display:flex;gap:16px;margin:24px 0">
  <a href="$./6.1 - Documentation and Knowledge Transfer" style="flex:1;border:1px solid #e0e0e0;border-radius:8px;padding:16px 20px;text-decoration:none;text-align:left">
    <span style="display:block;font-size:12px;color:#666;margin-bottom:4px">Previous</span>
    <span style="display:block;font-size:16px;font-weight:600;color:#1a5276">« Documentation and Knowledge Transfer</span>
  </a>
  <div style="flex:1"></div>
</div>

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>
